In [ ]:
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import regularizers
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D,Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ================================
# 1. CARGA Y PROCESAMIENTO DEL DATASET
# ================================

dataset_path = r"/content/drive/MyDrive/UCMerced_LandUse/Images"
nclases = 21
img_size = (256, 256)

X = []
y = []
clase_dict = {}

for idx, clase in enumerate(sorted(os.listdir(dataset_path))):
    clase_path = os.path.join(dataset_path, clase)
    if os.path.isdir(clase_path):
        clase_dict[idx] = clase
        for img_name in os.listdir(clase_path):
            img_path = os.path.join(clase_path, img_name)
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img)
                X.append(img_array)
                y.append(idx)
            except Exception as e:
                print(f"Error cargando imagen {img_path}: {e}")

X = np.array(X).astype('float32') / 255.0
y = np.array(y)

# División 80-10-10
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# One-hot encoding
y_train_cat = to_categorical(y_train, nclases)
y_val_cat = to_categorical(y_val, nclases)
y_test_cat = to_categorical(y_test, nclases)

print(f"Training data shape: {X_train.shape}, {y_train_cat.shape}")
print(f"Validation data shape: {X_val.shape}, {y_val_cat.shape}")
print(f"Test data shape: {X_test.shape}, {y_test_cat.shape}")

In [ ]:
# ================================
# 2. DEFINICIÓN DEL MODELO CNN
# ================================

cnn = Sequential([
    # Bloque 1
    Conv2D(64, (3, 3), activation='relu', padding='same',
           input_shape=(256, 256, 3),
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Bloque 2
    Conv2D(128, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Bloque 3
    Conv2D(256, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same',
           kernel_regularizer=regularizers.l2(0.0005)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Global Average Pooling en vez de Flatten
    GlobalAveragePooling2D(),

    # Capas densas
    Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(21, activation='softmax')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
cnn.compile(optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

# ================================
# 3. ENTRENAMIENTO DEL MODELO
# ================================

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=25,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=12,
    min_lr=0.00001
)

history = cnn.fit(
    datagen.flow(X_train, y_train_cat, batch_size=32),
    validation_data=(X_val, y_val_cat),
    epochs=500,
    callbacks=[early_stopping, reduce_lr]
)


In [ ]:
# ================================
# 4. EVALUACIÓN DEL MODELO
# ================================

train_loss, train_accuracy = cnn.evaluate(X_train, y_train_cat)
val_loss, val_accuracy = cnn.evaluate(X_val, y_val_cat)
test_loss, test_accuracy = cnn.evaluate(X_test, y_test_cat)

print(f"\nTraining Accuracy: {train_accuracy*100:.2f}%")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


In [ ]:
# ================================
# 5. GRÁFICAS DE ENTRENAMIENTO
# ================================

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()



In [ ]:
# ================================
# 6. MATRIZ DE CONFUSIÓN DATOS DE TESTEO
# ================================

y_pred = np.argmax(cnn.predict(X_test), axis=-1)
y_true = np.argmax(y_test_cat, axis=-1)

conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(15, 12))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(clase_dict.values()),
            yticklabels=list(clase_dict.values()))
plt.title('CONFUSION MATRIX TEST DATA - UC Merced Dataset')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# ================================
# 7. MATRIZ DE CONFUSIÓN DATOS DE VALIDACIÓN
# ================================

y_pred = np.argmax(cnn.predict(X_val), axis=-1)
y_true = np.argmax(y_val_cat, axis=-1)
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(15, 12))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(clase_dict.values()),
            yticklabels=list(clase_dict.values()))
plt.title('CONFUSION MATRIX VALIDATION DATA - UC Merced Dataset')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
# ================================
# 7. MATRIZ DE CONFUSIÓN DATOS DE ENTRENAMIENTO
# ================================

y_pred = np.argmax(cnn.predict(X_train), axis=-1)
y_true = np.argmax(y_train_cat, axis=-1)
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(15, 12))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(clase_dict.values()),
            yticklabels=list(clase_dict.values()))
plt.title('CONFUSION MATRIX TRAIN DATA - UC Merced Dataset')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()



In [ ]:
# ================================
# 7. PRECISIÓN POR CLASE
# ================================

print("\nAccuracy por clase:")
for i in range(nclases):
    mask = (y_true == i)
    if np.sum(mask) > 0:
        acc = accuracy_score(y_true[mask], y_pred[mask])
        print(f"{clase_dict[i]}: {acc*100:.2f}%")

In [ ]:
from sklearn.metrics import classification_report

y_pred = np.argmax(cnn.predict(X_val), axis=-1)
y_true = np.argmax(y_val_cat, axis=-1)

print(classification_report(
    y_true, y_pred,
    target_names=list(clase_dict.values())
))


In [ ]:
# ================================
# 8. MAPA DE SALIENCIA (EXPLICACIÓN DEL MODELO)
# ================================
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

def generar_mapa_saliencia(modelo, imagen, etiqueta_real, clases):
    """
    Genera y muestra el mapa de saliencia para una imagen dada,
    explicando qué regiones influyeron más en la predicción del modelo.
    """

    # Aseguramos que la imagen tenga el formato correcto (batch de 1)
    img_tensor = tf.convert_to_tensor(imagen[np.newaxis, ...])

    # Calculamos los gradientes respecto a la entrada
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predicciones = modelo(img_tensor)
        clase_predicha = tf.argmax(predicciones[0])
        score = predicciones[:, clase_predicha]

    # Gradientes del score con respecto a la imagen de entrada
    gradientes = tape.gradient(score, img_tensor)

    # Tomamos el valor absoluto y combinamos canales RGB (máximo)
    saliencia = tf.reduce_max(tf.abs(gradientes), axis=-1)[0]

    # Normalizamos entre 0 y 1
    saliencia = (saliencia - tf.reduce_min(saliencia)) / (tf.reduce_max(saliencia) + 1e-8)

    # Visualización
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(imagen)
    plt.title(f'Imagen Original\nClase real: {clases[etiqueta_real]}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(imagen)
    plt.imshow(saliencia, cmap='plasma', alpha=0.5)
    plt.title(f'Mapa de Saliencia\nPredicción: {clases[int(clase_predicha)]}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# ================================
# Ejemplo de uso del mapa de saliencia
# ================================

# Selecciona un índice del conjunto de prueba
indice = 50

imagen_ejemplo = X_test[indice]
etiqueta_real = np.argmax(y_test_cat[indice])

generar_mapa_saliencia(cnn, imagen_ejemplo, etiqueta_real, clase_dict)


Versión 1: Mejora visual del mapa de saliencia

In [ ]:
def generar_mapa_saliencia_mejorado(modelo, imagen, etiqueta_real):
    """
    Genera un mapa de saliencia con mejor visualización (tipo heatmap intensivo).
    """
    img_tensor = tf.convert_to_tensor(imagen[np.newaxis, ...])
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predicciones = modelo(img_tensor)
        clase_predicha = tf.argmax(predicciones[0])
        score = predicciones[:, clase_predicha]

    gradientes = tape.gradient(score, img_tensor)
    saliencia = tf.reduce_max(tf.abs(gradientes), axis=-1)[0]

    # Normalización con realce gamma
    saliencia = (saliencia - tf.reduce_min(saliencia)) / (tf.reduce_max(saliencia) + 1e-8)
    saliencia = tf.pow(saliencia, 0.5)  # realza áreas calientes

    # Conversión a RGB heatmap tipo GradCAM
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(imagen)
    plt.title(f'Clase real: {clase_dict[etiqueta_real]}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(imagen)
    plt.imshow(saliencia, cmap='jet', alpha=0.55)
    plt.title(f'Mapa de Saliencia Mejorado\nPredicción: {clase_dict[int(clase_predicha)]}')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
indice = 20
imagen_ejemplo = X_test[indice]
etiqueta_real = np.argmax(y_test_cat[indice])

# Versión mejorada (sin Grad-CAM):
generar_mapa_saliencia_mejorado(cnn, imagen_ejemplo, etiqueta_real)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def generar_mapa_saliencia_mejorado(modelo, imagen, etiqueta_real, clase_dict):
    """
    Genera un mapa de saliencia con mejor visualización (tipo heatmap intensivo).
    Retorna la imagen original y el mapa para graficarlos externamente.
    """
    img_tensor = tf.convert_to_tensor(imagen[np.newaxis, ...])
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predicciones = modelo(img_tensor)
        clase_predicha = tf.argmax(predicciones[0])
        score = predicciones[:, clase_predicha]

    gradientes = tape.gradient(score, img_tensor)
    saliencia = tf.reduce_max(tf.abs(gradientes), axis=-1)[0]

    # 🔥 Normalización con realce gamma (mejor contraste)
    saliencia = (saliencia - tf.reduce_min(saliencia)) / (tf.reduce_max(saliencia) + 1e-8)
    saliencia = tf.pow(saliencia, 0.5)

    return saliencia.numpy(), int(clase_predicha)


# =========================================
# 🔧 Mostrar una imagen por clase (2 por fila)
# =========================================

num_clases = len(clase_dict)
cols = 2
rows = int(np.ceil(num_clases))  # una clase por par (original + saliencia)

plt.figure(figsize=(10, 5 * rows))

for i in range(num_clases):
    # Encuentra el primer ejemplo de esa clase
    idx = np.where(np.argmax(y_test_cat, axis=1) == i)[0][0]
    imagen = X_test[idx]
    etiqueta_real = i

    # Generar mapa de saliencia
    saliencia, clase_predicha = generar_mapa_saliencia_mejorado(cnn, imagen, etiqueta_real, clase_dict)

    # --- Imagen original ---
    plt.subplot(rows, cols, 2*i + 1)
    plt.imshow(imagen)
    plt.title(f"Real: {clase_dict[etiqueta_real]}")
    plt.axis('off')

    # --- Imagen con heatmap ---
    plt.subplot(rows, cols, 2*i + 2)
    plt.imshow(imagen)
    plt.imshow(saliencia, cmap='jet', alpha=0.55)
    plt.title(f"Pred: {clase_dict[clase_predicha]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# === Función para generar mapa de saliencia mejorado ===
def generar_mapa_saliencia_mejorado(modelo, imagen, etiqueta_real, clase_dict):
    """
    Genera un mapa de saliencia tipo heatmap con mejor contraste visual.
    Devuelve el mapa y la clase predicha.
    """
    img_tensor = tf.convert_to_tensor(imagen[np.newaxis, ...])
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predicciones = modelo(img_tensor)
        clase_predicha = tf.argmax(predicciones[0])
        score = predicciones[:, clase_predicha]

    gradientes = tape.gradient(score, img_tensor)
    saliencia = tf.reduce_max(tf.abs(gradientes), axis=-1)[0]

    # Normalización + realce gamma (mejora visibilidad)
    saliencia = (saliencia - tf.reduce_min(saliencia)) / (tf.reduce_max(saliencia) + 1e-8)
    saliencia = tf.pow(saliencia, 0.5)

    return saliencia, int(clase_predicha)

# === 📂 Ruta personalizada en tu Google Drive ===
output_dir = "/content/drive/MyDrive/Saliency_Maps"
os.makedirs(output_dir, exist_ok=True)

# === Generar mapa de saliencia para una imagen por clase ===
num_clases = len(clase_dict)
for i in range(num_clases):
    indices = np.where(np.argmax(y_test_cat, axis=1) == i)[0]
    if len(indices) == 0:
        continue  # Saltar si no hay imágenes de esa clase
    idx = indices[0]

    imagen = X_test[idx]
    etiqueta_real = i

    # Generar mapa
    saliencia, clase_predicha = generar_mapa_saliencia_mejorado(cnn, imagen, etiqueta_real, clase_dict)

    # Crear figura
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(imagen)
    plt.title(f"Real: {clase_dict[etiqueta_real]}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(imagen)
    plt.imshow(saliencia, cmap='jet', alpha=0.55)
    plt.title(f"Pred: {clase_dict[clase_predicha]}")
    plt.axis('off')

    plt.tight_layout()

    # Guardar imagen
    filename = f"{clase_dict[etiqueta_real]}_saliency.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.close()

print(f"✅ Mapas de saliencia guardados en tu Drive en: {output_dir}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def generar_mapa_saliencia_mejorado(modelo, imagen, etiqueta_real, clase_dict):
    """
    Genera un mapa de saliencia con mejor visualización (tipo heatmap intensivo).
    Retorna la imagen original y el mapa para graficarlos externamente.
    """
    img_tensor = tf.convert_to_tensor(imagen[np.newaxis, ...])
    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        predicciones = modelo(img_tensor)
        clase_predicha = tf.argmax(predicciones[0])
        score = predicciones[:, clase_predicha]

    gradientes = tape.gradient(score, img_tensor)
    saliencia = tf.reduce_max(tf.abs(gradientes), axis=-1)[0]

    # 🔥 Normalización con realce gamma (mejor contraste)
    saliencia = (saliencia - tf.reduce_min(saliencia)) / (tf.reduce_max(saliencia) + 1e-8)
    saliencia = tf.pow(saliencia, 0.5)

    return saliencia.numpy(), int(clase_predicha)


# =========================================
# 🔧 Mostrar una imagen aleatoria por clase (2 por fila)
# =========================================

num_clases = len(clase_dict)
cols = 2
rows = num_clases  # 1 fila por clase (original + saliencia)

plt.figure(figsize=(10, 4 * rows))

for i in range(num_clases):
    # 🔁 Seleccionar un índice aleatorio de esa clase
    indices_clase = np.where(np.argmax(y_test_cat, axis=1) == i)[0]
    if len(indices_clase) == 0:
        continue  # por si no hay muestras de esa clase en test
    idx = np.random.choice(indices_clase)  # 👈 aquí ocurre la aleatoriedad

    imagen = X_test[idx]
    etiqueta_real = i

    # Generar mapa de saliencia
    saliencia, clase_predicha = generar_mapa_saliencia_mejorado(cnn, imagen, etiqueta_real, clase_dict)

    # --- Imagen original ---
    plt.subplot(rows, cols, 2*i + 1)
    plt.imshow(imagen)
    plt.title(f"Real: {clase_dict[etiqueta_real]}")
    plt.axis('off')

    # --- Imagen con heatmap ---
    plt.subplot(rows, cols, 2*i + 2)
    plt.imshow(imagen)
    plt.imshow(saliencia, cmap='jet', alpha=0.55)
    plt.title(f"Pred: {clase_dict[clase_predicha]}")
    plt.axis('off')

plt.tight_layout()
plt.show()
